In [4]:
import numpy as np
import numpy.matlib 
import math
import matplotlib.pyplot as plt
import scipy.stats

# Inspired by youtube channel D&D Optimised (now d4: D&D Deep Dive)

## Calculate average damage

### Source: D&D Optimised
https://www.youtube.com/watch?v=nk2SIaGXSYo

"The basic formula for calculating average damage is as follows:				
((Average Weapon Damage + Applicable per-hit Damage Modifiers)*(% chance to hit))+((Average Dice Damage on attack)*(% chance to crit))+any once per round damage													
													
Keep in mind that if you have multiple attacks, you need to run the above for each attack - you can simply multiply everything between the outermost parentheses by the # of attacks *if* those attacks are identical,													
but in the case of someone using a Polearm Master Feat, for example, they are not, as the regular attacks would use a D10 for the weapon, wheras the bonus action attack would use a D4 for the weapon."													

### Source: https://www.dndbeyond.com/sources/phb/combat#MakinganAttack
"To make an attack roll, roll a d20 and add the appropriate modifiers. If the total of the roll plus modifiers equals or exceeds the target's Armor Class (AC), the attack hits." 


## Detailed calculations

### Definitions
[roll]: Average Dice Roll
OAC: Opponents Armour Class

### Chance to hit and chance to miss

P(My hit was greater or equal than the Opponents AC) = P([roll]+ hit bonus >= OAC) => P([roll]>= OAC- hit bonus)

P(hit)=P([rol] >= OAC -hit bonus) = [ 20 -(OAC- hit bonus)+1]/20 = 1 -(OAC- hit bonus -1)/20 = 1 - P(miss)

P(miss) = (OAC- hit bonus -1)/20

### Chance to hit with advantage
P (hit on first roll but not on the second)+ P (hit on the second roll but not on the first)+ P(hit on both rolls)=  2  * [ 1 -(OAC- hit bonus -1)/20] * [ (OAC- hit bonus -1)/20]+ [ 1 -(OAC- hit bonus -1)/20] **2 (see also binomial distribution).

The shorter way is to calculate "1 - the chance of missing both rolls". 

P (miss both rolls) = [P(miss)] = [(OAC- hit bonus -1)/20]**2

P( hit with an advantage) = 1 - [(OAC- hit bonus -1)/20]**2

### Chance to hit with advantage and Elven Accuracy (or Lucky)
Elven accuracy, i.e roll one more time: 1- [(OAC- hit bonus -1)/20]**3

### Critical hit  (minimum chance to hit)
Crit hit (natural 20): p(critit)= 1/20

### Critical hit with advantage
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**2

### Minimum chance to hit with advantage and Elven Accuracy (or Lucky)
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**3

### Minimum chance to hit with Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)

In [5]:
1- (1-2/20)

0.09999999999999998

### Minimum chance to hit with advantage, Elven Accuracy (or Lucky) and Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)**3

In [6]:
1- (1-2/20)**3

0.2709999999999999

One way to obtain advantage as a Warlock is to get Devil's Sight and Cast Darkness on oneself (when multiclass with e.g. fighter, one may use the blind fighting style instead of Devil's Sight).

##  Calculating the Probablility of critical hits (rounded to 5 decimal digits).

In [7]:
import dnd_optimisation as dndopt # this is a python file containing the calcuations for critical hits

In [8]:
import importlib  #This is because I am making changes to the file all the time ;) 
importlib.reload(dndopt)  

<module 'dnd_optimisation' from '/Users/eleni/Python Scripts/DnD/dnd_optimisation.py'>

#### Setting advantage, luck_point (related to the lucky feat) and elven_accuracy to false gives the probabity of a natural critical hit (natural 20) 1/20

In [9]:
advantage=False; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [10]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.05

#### Only advantage

In [11]:
advantage=True; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [12]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.0975

#### Advantage and lucky

In [13]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=False

In [14]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.14263

#### Hexblade curse with advantage

In [15]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=True

In [16]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.271

# Comparing different builds

### Assuming the following Ability Scores

In [17]:
AbilityScores= numpy.array([19, 14, 12, 16, 8, 18])

In [18]:
Abilities=['STR', 'DEX', 'CON','INT','WIS', 'CHA']

### Calculate the Ability Score Modifiers

In [19]:
Modifiers = np.floor((AbilityScores - 10)/2)

In [20]:
Modifiers=Modifiers.astype(int)

### What is the Ability Score for Charisma?

In [21]:
AbilityScores[Abilities.index('CHA')]

18

### What is the Modifier for Charisma?

In [22]:
Modifiers[Abilities.index('CHA')]

4

### Define the Character's Level

In [23]:
Level=8;

### Calculate the Proficiency Bonus

In [24]:
ProficiencyBonus=1+np.ceil(Level/4)

In [25]:
ProficiencyBonus=ProficiencyBonus.astype(int)

### Calculate Hit for Spells Attacks casted with Charisma

In [26]:
OtherBonus=1; #Rod of pact keeper

In [27]:
SpellHit=Modifiers[Abilities.index('CHA')]+ProficiencyBonus+OtherBonus

In [28]:
SpellHit

8

### Calculate Hit for Weapon Attacks with Strength

In [29]:
WeaponBonus=1; #Magical +1 weapon

In [30]:
MeleeHit=Modifiers[Abilities.index('STR')]+ProficiencyBonus+WeaponBonus

In [31]:
MeleeHit

8

In [32]:
### Calculate Hit for Weapon Attacks with Dexterity

In [33]:
DexMeleeHit=Modifiers[Abilities.index('DEX')]+ProficiencyBonus+WeaponBonus

In [34]:
DexMeleeHit

6

### The Opponent's Armor Class is:

In [35]:
OAC=15

## Calculate the damage of Eldrich Blast

### Probability to hit an Opponent

#### Probability to hit the Opponent (for 1 dice roll)

In [36]:
PrHit= 1 -(OAC- SpellHit -1)/20 

In [37]:
PrHit

0.7

#### using a function to calculate the probability 
####  probability_of__hitting_opponent(opponent_AC,modifiers,advantage,luck_point,elven_accuracy)

In [38]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,0,0,0)

0.7

#### Probability to miss the Opponent (for 1 dice roll)

In [39]:
PrMiss=(OAC- SpellHit -1)/20 

In [40]:
PrMiss

0.3

#### Probability to hit the opponent with advantage (Ad=2)

In [41]:
Ad=2

In [42]:
PrHitA=1-(PrMiss)**(Ad)

In [43]:
PrHitA

0.91

#### testing function

In [44]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,1,0,0)

0.91

#### Number of Elrdich Blast Rays

In [45]:
Beams=min(math.floor(Level/5)+1,4)

In [46]:
Beams

2

#### In that case the average damage (per ray) will be (with Agonizing Blast) 1d10+ Charisma Modifier

In [47]:
AverageAgonisingBlastDamage=np.mean(range(1,11))

In [48]:
AverageAgonisingBlastDamage

5.5

In [49]:
AverageDamagePerBeam=AverageAgonisingBlastDamage+Modifiers[Abilities.index('CHA')]

In [50]:
AverageDamagePerBeam

9.5

Additional modifier for Hex spell

In [51]:
HexAverageDamage=3.5

### Average damage, taking into account critical hits, parametrised by the number of rays

For two rays only the formula would be 

 2 x Probability that only one ray will hit x damage + Probability that both rays will hit x 2 x damage + 2 x Probability that only one dice will crit hit x damage  + Probability that both dices will hit x 2 x damage 

However, the modifier is only taken into account once

In [52]:
Critit=1/20;

In [53]:
2 * PrHit * PrMiss * AverageDamagePerBeam+ 2* (Critit) *(1-Critit) * AverageAgonisingBlastDamage+ PrHit**2 * 2* AverageDamagePerBeam + 2* (Critit) *(Critit) * AverageAgonisingBlastDamage

13.849999999999998

In order to easily generalise to higher number of Beams we use the binomial distribution. we check for 2 Beams to see the same result as above.

Without taking into account Crit Hit

In [54]:
ADNCR = scipy.stats.binom.pmf(1,2,PrHit)*AverageDamagePerBeam + scipy.stats.binom.pmf(2,2,PrHit)* 2* AverageDamagePerBeam 

In [55]:
ADNCR

13.3

Calculating Crit Hit

In [56]:
ACHD= scipy.stats.binom.pmf(1,2,Critit)*AverageAgonisingBlastDamage + scipy.stats.binom.pmf(2,2,Critit)* 2* AverageAgonisingBlastDamage

In [57]:
ACHD

0.5500000000000002

Adding the two together

In [58]:
TAD=ADNCR+ACHD

In [59]:
TAD

13.850000000000001

#### Calculating for arbitrary number of Beams

In [60]:
TotalDamage=0;

In [61]:
indexBeam = 1
while indexBeam < (Beams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,Beams,PrHit)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,Beams,Critit)*indexBeam*AverageAgonisingBlastDamage
    indexBeam += 1

In [62]:
TotalDamage

13.850000000000001

### Average damage with  Advantage

In [63]:
n=1

In [64]:
TotalBeams=n*Beams

In [65]:
CrititA=1-(1-1/20)**2

In [66]:
2 * PrHitA * (1-PrHitA) * AverageDamagePerBeam+ 2* (CrititA) *(1-CrititA) * AverageAgonisingBlastDamage+ PrHitA**2 * 2* AverageDamagePerBeam + 2* (CrititA) *(CrititA) * AverageAgonisingBlastDamage

18.3625

Confirming the result using the binomial distribution function

In [67]:
TotalDamage=0

In [68]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*AverageAgonisingBlastDamage
    indexBeam += 1

In [69]:
TotalDamage

18.362500000000004

### Average damage with Advantage and Hex

In [70]:
n=1

In [71]:
TotalBeams=n*Beams

In [72]:
TotalDamage=0

In [73]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*(AverageAgonisingBlastDamage+HexAverageDamage)
    indexBeam += 1

In [74]:
TotalDamage

25.415

### Average damage with  n repetitions (e.g. Action Surge Metamagic quickened spell n=2)

In [75]:
n=2;

In [76]:
TotalBeams=n*Beams

In [77]:
TotalDamage=0

In [78]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHit)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,Critit)*indexBeam*AverageAgonisingBlastDamage
    indexBeam += 1

In [79]:
TotalDamage

27.700000000000006

### Average damage with  n repetitions  (e.g. Action surge n=2) and Advantage

For rolls with advantage I need to use in the formula the probability of hitting the target with advantage

In [80]:
n=2;

In [81]:
TotalBeams=n*Beams

In [82]:
TotalDamage=0

In [83]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*AverageAgonisingBlastDamage
    indexBeam += 1

In [84]:
  TotalDamage

36.725

#### Testing the function average_damage(repetitions,average_damage_per_hit,damage_modifier,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [85]:
damage_modifier=Modifiers[Abilities.index('CHA')]

In [86]:
repetitions=4;average_damage_per_hit=AverageAgonisingBlastDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [87]:
dndopt.average_damage(repetitions,average_damage_per_hit,damage_modifier,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

36.725

### Average damage with  n repetitions  (e.g. Action surge n=2), Advantage and Hex

In [88]:
n=2;

In [89]:
TotalBeams=n*Beams

In [90]:
TotalBeams

4

In [91]:
TotalDamage=0

In [92]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*(AverageAgonisingBlastDamage+HexAverageDamage)
    indexBeam += 1

In [93]:
TotalDamage

50.83

#### Testing the function average_damage(repetitions,average_damage_per_hit,damage_modifier,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [94]:
repetitions=4;average_damage_per_hit=AverageAgonisingBlastDamage+HexAverageDamage;damage_modifier=Modifiers[Abilities.index('CHA')];opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [95]:
dndopt.average_damage(repetitions,average_damage_per_hit,damage_modifier,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

50.83

### Without action surge, only advantage, Hex and two eldrich blast rays (repetitions=2). 

In [96]:
repetitions=Beams;average_damage_per_hit=AverageAgonisingBlastDamage+HexAverageDamage;damage_modifier=Modifiers[Abilities.index('CHA')];opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [97]:
dndopt.average_damage(repetitions,average_damage_per_hit,damage_modifier,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

25.415

### With Action surge and spirit shroud so that I can use the celestial warlock's bonus to radient damage.

In [98]:
spirit_shroud=4.5

In [99]:
radient_damage_bonus=Modifiers[Abilities.index('CHA')]# since it is applied only to the first beam, multiply with factor 1/2 to calculate average damage

In [100]:
repetitions=4; average_damage_per_hit=AverageAgonisingBlastDamage+spirit_shroud+radient_damage_bonus/2;damage_modifier=Modifiers[Abilities.index('CHA')];opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [101]:
dndopt.average_damage(repetitions,average_damage_per_hit,damage_modifier,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

62.92

#### Assuming warlock with no radient bonus and advantage

In [102]:
repetitions=4; average_damage_per_hit=AverageAgonisingBlastDamage+spirit_shroud;damage_modifier=Modifiers[Abilities.index('CHA')];opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [103]:
dndopt.average_damage(repetitions,average_damage_per_hit,damage_modifier,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

54.86